In [14]:
import matplotlib.pyplot as plt
# import numpy as np
from PIL import Image

# import torch
from torchvision import transforms

def pil_loader(path):
	return Image.open(path).convert('RGB')

tfs = transforms.Compose([
	transforms.Resize((256, 256)),
	transforms.ToTensor(),
	transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
	transforms.Grayscale(num_output_channels=1),
])

tfs_to_tensor = transforms.Compose([
	# transforms.Resize((256, 256)),
	transforms.ToTensor(),
	# transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
	# transforms.Grayscale(num_output_channels=1),
])

In [3]:
lr_image = '../matter-density-superresolution/test/lr/1-0.png'
hr_image = '../matter-density-superresolution/test/hr/1-0.png'
half_mask = 'half.png'

# Load images using numpy
lr = tfs(pil_loader(lr_image))[0]
hr = tfs(pil_loader(hr_image))[0]
half_mask = tfs_to_tensor(pil_loader(half_mask))[0]

# # Plot images
plt.figure(figsize=(10, 10))
plt.subplot(1, 3, 1)
plt.imshow(lr, cmap='gray')
plt.title('Low Resolution')
# plt.subplot(1, 3, 2)
# plt.imshow(hr, cmap='gray')
# plt.title('High Resolution')
# plt.subplot(1, 3, 3)
# plt.imshow(half_mask, cmap='gray')
# plt.title('Half Mask')

# plt.show()

Text(0.5, 1.0, 'Low Resolution')

: 

: 

In [ ]:
# %run run_inpaint.py --lr-img-path ../matter-density-superresolution/test/lr/1-0.png --hr-img-path ../matter-density-superresolution/test/hr/1-0.png --mask-path half.png --num-inference-steps 100 --jump-length 10 --jump-n-sample 10 -c config/superresolution.json

In [ ]:
# half_mask = np.array(half_mask)[:,:,0] // np.max(np.array(half_mask))


In [ ]:
# inpaint_img = x_t.squeeze().detach().cpu().numpy()
inpaint_img = tfs(pil_loader('output/repaint.png'))[0]

plt.figure(figsize=(10, 10))
plt.subplot(1, 3, 1)
plt.imshow(hr*half_mask, cmap='gray')
plt.title('Masked Image')

plt.subplot(1, 3, 2)
plt.imshow(inpaint_img, cmap='gray')
plt.title('Inpainted Image')

plt.subplot(1, 3, 3)
plt.imshow(hr, cmap='gray')
plt.title('Ground Truth')
plt.show()

In [15]:
# Stitch together
import torch
from os import mkdir, path
from shutil import rmtree

lr_full_path_0 = '../matter-density-superresolution/full/lr/0.png'
hr_full_path_0 = '../matter-density-superresolution/full/hr/0.png'

lr_full_0 = tfs_to_tensor(pil_loader(lr_full_path_0))[0]
hr_full_0 = tfs_to_tensor(pil_loader(hr_full_path_0))[0]

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(lr_full_0, cmap='gray')
plt.clim(0, 1)
plt.title('Low Resolution')

plt.subplot(1, 2, 2)
plt.imshow(hr_full_0, cmap='gray')
plt.clim(0, 1)
plt.title('High Resolution')

def split_patches(directory, lr_full, hr_full, i, size, stride):
	if path.isdir(directory):
		rmtree(directory)
	mkdir(directory)
	mkdir(f'{directory}/hr')
	mkdir(f'{directory}/lr')
	mkdir(f'{directory}/mask')

	fnames = []

	mask_full = torch.zeros_like(lr_full)
	
	for x_start in range(0, lr_full.shape[0] - stride, stride):
		for y_start in range(0, lr_full.shape[1] - stride, stride):
			x_end = x_start + size
			y_end = y_start + size

			fname = f'{i}-x-{x_start}-y-{y_start}'
			fnames.append(fname)

			img = Image.fromarray((hr_full[x_start:x_end, y_start:y_end] * 255).type(torch.uint8).numpy(), 'L')
			img.save(f'{directory}/hr/{fname}.png')

			img = Image.fromarray((lr_full[x_start:x_end, y_start:y_end] * 255).type(torch.uint8).numpy(), 'L')
			img.save(f'{directory}/lr/{fname}.png')

			mask = Image.fromarray((mask_full[x_start:x_end, y_start:y_end] * 255).type(torch.uint8).numpy(), 'L')
			mask.save(f'{directory}/mask/{fname}.png')
			mask_full[x_start:x_end, y_start:y_end] = 1
		
	np.savetxt(f'{directory}/flist.txt', np.array(fnames, dtype=str), '%s', encoding='utf-8')
	print(f'Generated {len(fnames)} images for {directory} dataset.')

split_patches('patches', lr_full_0, hr_full_0, 0, 256, 256 // 2)

Generated 49 images for patches dataset.


: 

: 

In [2]:
import numpy as np

images = [i for i in np.genfromtxt('patches/flist.txt', dtype=str, encoding='utf-8')]

cmds = []
for i, image in enumerate(images):
	lr_img_path = f'patches/lr/{image}.png'
	hr_img_path = f'patches/generated_hr/{image}.png'
	mask_path = f'patches/mask/{image}.png'
	output_path =f'patches/output/{image}.png'

	parts = image.split('-')
	x = int(parts[2])
	y = int(parts[4])

	cmds.append(f'python compile_patches.py -x {x} -y {y}')
	if i == 0:
		cmd = f'python superresolve.py --lr-img-path {lr_img_path} --hr-img-path {hr_img_path} -o {output_path} -c config/superresolution.json'
	else:
		cmd = f'python run_inpaint.py --lr-img-path {lr_img_path} --hr-img-path {hr_img_path} --mask-path {mask_path} -o {output_path} --num-inference-steps 100 --jump-length 10 --jump-n-sample 10 -c config/superresolution.json'
	cmds.append(cmd)
	
	# break
	# if i > 2:
	# 	break

cmds.append('python compile_patches.py')
np.savetxt('generate_patches.bat', np.array(cmds, dtype=str), '%s', encoding='utf-8')